### **1. 张量（Tensor）**
- **定义**：PyTorch中的基本数据结构，是多维数组的扩展，支持GPU加速计算和自动微分。
- **关键点**：
  - 类似NumPy的`ndarray`，但可通过`.to(device)`迁移到GPU或者其它加速卡（昇腾、寒武纪MLU）。
  - 创建方式：`torch.tensor(data)`, `torch.zeros()`, `torch.rand()`等。或者从`List`和`np.arrary`转换，`torch.tensor(data_list)`或`torch.from_numpy(data_np)`。
  - 支持数学运算（如加减乘除、矩阵乘法`@`或`torch.matmul()`）。
  - 形状操作：`view()`（需连续内存）、`reshape()`（自动处理非连续内存）。
  - 类型转换：`float()`、`long()`、`int()`、`bool()`。

In [2]:
import torch
# 创建一个形状为(2, 2)的张量，数据类型为float32，可求导。
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, requires_grad=True,device="cpu")
print("x 形状:", x.shape)
print("x 数据类型:", x.dtype)
y = x.view(1, 4)
print("y 形状:", y.shape)
z = x.int()
print("z 数据类型:", z.dtype)
w = x.to("cuda")
print("w 设备:", w.device)
x2 = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
x3 = x2.reshape(2, 2)
print("x2 形状:", x2.shape)
print("x3 形状:", x3.shape)
x4 = x3.transpose(0, 1)
print("x4 形状:", x4.shape)
x5 = x2.unsqueeze(0)
print("x5 形状:", x5.shape)
x6 = x2.squeeze(0)
print("x6 形状:", x6.shape)
x7 = torch.cat([x, x3], dim=0)
print("x7 形状:", x7.shape)
print("x7 元素:", x7)

x 形状: torch.Size([2, 2])
x 数据类型: torch.float32
y 形状: torch.Size([1, 4])
z 数据类型: torch.int32
w 设备: cuda:0
x2 形状: torch.Size([4])
x3 形状: torch.Size([2, 2])
x4 形状: torch.Size([2, 2])
x5 形状: torch.Size([1, 4])
x6 形状: torch.Size([4])
x7 形状: torch.Size([4, 2])
x7 元素: tensor([[1., 2.],
        [3., 4.],
        [1., 2.],
        [3., 4.]], grad_fn=<CatBackward0>)


### **2. 自动微分（Autograd）**
- **定义**：PyTorch通过`autograd`模块自动计算梯度，用于反向传播。
- **关键点**：
  - `requires_grad=True`：标记需要计算梯度的张量。
  - `backward()`：从标量张量触发梯度计算，结果存储在`.grad`属性中。
  - 计算图：动态构建，每次前向传播生成新图。
  - 上下文管理器：`torch.no_grad()`禁用梯度跟踪（如推理时）。

In [5]:
x = torch.tensor([1., 2.], requires_grad=True)
y = torch.tensor([3., 4.], requires_grad=True)
z = x * y
print(z.shape)
print(z)
z.sum().backward() #计算梯度
print("x.grad:", x.grad) #输出梯度值
print("y.grad:", y.grad) #输出梯度值

torch.Size([2])
tensor([3., 8.], grad_fn=<MulBackward0>)
x.grad: tensor([3., 4.])
y.grad: tensor([1., 2.])


在下方的代码当中，我们构建了一个简单的神经网络。
并使用 **均方误差（MSELoss）** 计算损失。我们可以用 **LaTeX** 语言来表示前向传播和反向传播（梯度计算）的公式。  

---

### **1. 前向传播**
设：
- **输入** $ x \in \mathbb{R}^{bs\times 3} $，其中 $ bs $ 是批量大小
- **第一层权重** $ W_1 \in \mathbb{R}^{5 \times 3} $, **第一层偏置** $ b_1 \in \mathbb{R}^5 $
- **第二层权重** $ W_2 \in \mathbb{R}^{1 \times 5} $, **第二层偏置** $ b_2 \in \mathbb{R}^1 $
- **ReLU 激活函数** $ \sigma(x) = \max(0, x) $

则：

$h_1 = W_1 x + b_1 \\$
$a_1 = \sigma(h_1) = \max(0, h_1)\\$
$h_2 = W_2 a_1 + b_2\\$
$y_{\text{pred}} = h_2\\$

### **2. 损失函数**
损失使用 **均方误差（MSE）**：
$\mathcal{L} = (y_{\text{pred}} - y_{\text{true}})^2\\$

---

### **3. 反向传播**
#### **计算梯度**
1. **损失对最终输出** $ h_2 $ 的导数：
   $\\\frac{\partial \mathcal{L}}{\partial h_2} = 2 (h_2 - y_{\text{true}})\\$

2. **对第二层权重 $ W_2 $ 和偏置 $ b_2 $ 计算梯度**：
   $\\
   \frac{\partial \mathcal{L}}{\partial W_2} = \frac{\partial \mathcal{L}}{\partial h_2} \cdot \frac{\partial h_2}{\partial W_2}
   = 2 (h_2 - y_{\text{true}}) \cdot a_1^T
   \\$
   $
   \frac{\partial \mathcal{L}}{\partial b_2} = \frac{\partial \mathcal{L}}{\partial h_2} \cdot \frac{\partial h_2}{\partial b_2}
   = 2 (h_2 - y_{\text{true}})
   $

3. **对第一层激活 $ a_1 $ 计算梯度**：
   $
\\
   \frac{\partial \mathcal{L}}{\partial a_1} = \frac{\partial \mathcal{L}}{\partial h_2} \cdot W_2
   = 2 (h_2 - y_{\text{true}}) W_2
   $

4. **对第一层的输入 $ h_1 $ 计算梯度（考虑 ReLU）**：
   $
   \\
   \frac{\partial \mathcal{L}}{\partial h_1} = \frac{\partial \mathcal{L}}{\partial a_1} \odot \sigma'(h_1)\\$
   其中
   $
   \sigma'(h_1) =
   \begin{cases}
   1, & h_1 > 0 \\
   0, & h_1 \leq 0
   \end{cases}
   \\$
   因此：
   $
   \frac{\partial \mathcal{L}}{\partial h_1} = \left(2 (h_2 - y_{\text{true}}) W_2\right) \odot \mathbb{1}(h_1 > 0)
   $

5. **对第一层权重 $ W_1 $ 和偏置 $ b_1 $ 计算梯度**：
   $\\
   \frac{\partial \mathcal{L}}{\partial W_1} = \frac{\partial \mathcal{L}}{\partial h_1} \cdot x
   \\$
   $
   \frac{\partial \mathcal{L}}{\partial b_1} = \frac{\partial \mathcal{L}}{\partial h_1}
   $

---

### **最终梯度更新**
在反向传播过程中，PyTorch 会自动计算这些梯度，并用于 **梯度下降优化**：
$
W_i = W_i - \eta \frac{\partial \mathcal{L}}{\partial W_i}
$
$
b_i = b_i - \eta \frac{\partial \mathcal{L}}{\partial b_i}
$
其中 $ \eta $ 是学习率。

---

这样，你的神经网络就完成了一次前向传播、损失计算和反向传播的完整数学过程。🚀

In [40]:
Layer_1 = torch.nn.Linear(3, 5)
Relu = torch.nn.ReLU()
Layer_2 = torch.nn.Linear(5, 1)
Loss_func = torch.nn.MSELoss()

# 定义输入
x = torch.randn((1,3))
print(x.shape)
h1 = Layer_1(x)
print("h1:", h1)
a1 = Relu(h1)
print("a1:", a1)
h2 = Layer_2(a1)
print("h2:", h2)

# 定义输出
y_true = torch.tensor([[1]], dtype=torch.float32)
print("y_true:", y_true)
y_pred = h2
print("y_pred:", y_pred)

# 计算损失
loss = Loss_func(y_pred, y_true)
print("loss:", loss)
# 反向传播
loss.backward()
print()
print()
print("------Gradient of Layer_2------")


print("Auto Layer_2.weight.grad:", Layer_2.weight.grad)
print("Manual Layer_2.weight.grad:", 2 * (y_pred - y_true) * a1.t())
print("Auto Layer_2.bias.grad:", Layer_2.bias.grad)
print("Manual Layer_2.bias.grad:", 2 * (y_pred - y_true))

print()
print()
print("------Gradient of Layer_1------")

h1_grad = 2 * (y_pred - y_true) * Layer_2.weight *h1.gt(0)
print("First Row of Auto Layer_1.weight.grad:", Layer_1.weight.grad[1,:])
print("First Row of Manual Layer_1.weight.grad:", (h1_grad.t()*x)[1,:])
print("Auto Layer_1.bias.grad:", Layer_1.bias.grad)
print("Manual Layer_1.bias.grad:", h1_grad.sum(0))


torch.Size([1, 3])
h1: tensor([[-0.7639,  0.5218,  1.3314, -0.7480,  0.2545]],
       grad_fn=<AddmmBackward0>)
a1: tensor([[0.0000, 0.5218, 1.3314, 0.0000, 0.2545]], grad_fn=<ReluBackward0>)
h2: tensor([[-0.5780]], grad_fn=<AddmmBackward0>)
y_true: tensor([[1.]])
y_pred: tensor([[-0.5780]], grad_fn=<AddmmBackward0>)
loss: tensor(2.4900, grad_fn=<MseLossBackward0>)


------Gradient of Layer_2------
Auto Layer_2.weight.grad: tensor([[ 0.0000, -1.6468, -4.2019,  0.0000, -0.8033]])
Manual Layer_2.weight.grad: tensor([[-0.0000],
        [-1.6468],
        [-4.2019],
        [-0.0000],
        [-0.8033]], grad_fn=<MulBackward0>)
Auto Layer_2.bias.grad: tensor([-3.1559])
Manual Layer_2.bias.grad: tensor([[-3.1559]], grad_fn=<MulBackward0>)


------Gradient of Layer_1------
First Row of Auto Layer_1.weight.grad: tensor([-0.1434, -1.4336,  0.5011])
First Row of Manual Layer_1.weight.grad: tensor([-0.1434, -1.4336,  0.5011], grad_fn=<SliceBackward0>)
Auto Layer_1.bias.grad: tensor([ 0.0000, -0.